In [6]:
%load_ext autoreload
%autoreload 2

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
import logging
import os

os.makedirs('logs', exist_ok=True)
logging.basicConfig(filename='./logs/Illuminate_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Illuminate Operations Logging Instance')

def log_start():
    logging.info('\n\n-------------New Illuminate Operations Logging Instance')

def get_assessment_results(save_path):
    log_start()
    try:
        access_token, expires_in = get_access_token()

        assessments_df, assessment_id_list = get_all_assessments_metadata(access_token)
        assessment_id_list = assessment_id_list[:10] #for testing
        missing_ids_from_metadata = ['114845', '141498'] # Add assessments that are not present in assessements metadata
        assessment_id_list = list(set(assessment_id_list + missing_ids_from_metadata))
    
        # Fetch assessment results
        test_results_group, log_results_group = loop_through_assessment_scores(access_token, assessment_id_list, 'Group')
        test_results_standard, log_results_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'Standard')
        test_results_no_standard, log_results_no_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'No_Standard')
        
        test_results_combined = bring_together_test_results(test_results_no_standard, test_results_standard)
        test_results_view = create_test_results_view(test_results_combined, '24-25') #add in grade level col, string matching
        logging.info("Assessment results fetched and processed.")
        
        os.makedirs(save_path, exist_ok=True)

        # prior_year_file_path = '/home/g2015samtaylor/backups/illuminate'
        # test_results_group = append_prior_year(prior_year_file_path, test_results_group,  'assessment_results_group_2324.csv')
        # test_results_view = append_prior_year(prior_year_file_path, test_results_view,  'assessment_results_view_2324.csv')
        # test_results_combined = append_prior_year(prior_year_file_path, test_results_combined,  'assessment_results_combined_2324.csv')

        send_to_local(save_path, test_results_group, 'assessment_results_group_2425.csv')
        send_to_local(save_path, test_results_view, 'assessment_results_view_2425.csv')
        send_to_local(save_path, test_results_combined, 'assessment_results_combined_2425.csv')


    except Exception as e:
        logging.error(f"Error fetching assessment results: {e}")
        raise AirflowException("Failed to fetch and process assessment results")

save_path = '/home/g2015samtaylor/illuminate'
get_assessment_results(save_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The length of the ID_list is 12
The length of the ID_list is 12
The length of the ID_list is 12


In [1]:
%load_ext autoreload
%autoreload 2

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
import logging
import os

os.makedirs('logs', exist_ok=True)
logging.basicConfig(filename='./logs/Illuminate_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Illuminate Operations Logging Instance')




access_token, expires_in = get_access_token()

assessments_df, assessment_id_list = get_all_assessments_metadata(access_token)

In [5]:
assessment_id_list[:10]

['115265',
 '115266',
 '114855',
 '115001',
 '67540',
 '67539',
 '62294',
 '62295',
 '67544',
 '67543']

In [28]:
pd.read_csv('/home/g2015samtaylor/backups/illuminate/assessment_results_combined_2324.csv', encoding='ISO-8859-1')

/tmp/ipykernel_1285241/3785609462.py:1 DtypeWarning: Columns (2,4,20) have mixed types. Specify dtype option on import or set low_memory=False.

,assessment_id,title,academic_benchmark_guid,standard_code,standard_description,local_student_id,first_name,last_name,middle_name,date_taken,...,points_possible,percent_correct,performance_band_level,performance_band_label,mastered,__count,Standard_No_Standard,last_update,version,version_label
0,94608,Grade 6 Science: Unit 2 Traits and Reproduction,186E87F2-A70F-11E2-85F4-D4629DFF4B22,MS-LS3-1,Develop and use a model to describe why struct...,43939,Penelope,Alderete,NaN,2023-11-13,...,6,50,1,Does Not Meet Expectations,f,224,Standard,2024-12-16,NaN,NaN
1,94608,Grade 6 Science: Unit 2 Traits and Reproduction,186E87F2-A70F-11E2-85F4-D4629DFF4B22,MS-LS3-1,Develop and use a model to describe why struct...,43043,Oliver,Ramirez,NaN,2023-11-13,...,6,67,2,Approaching Expectations,f,224,Standard,2024-12-16,NaN,NaN
2,94608,Grade 6 Science: Unit 2 Traits and Reproduction,186FC8B0-A70F-11E2-85F4-D4629DFF4B22,MS-LS3-2,Develop and use a model to describe why asexua...,44433,Nickolas,Perez,NaN,2023-11-13,...,9,56,1,Does Not Meet Expectations,f,224,Standard,2024-12-16,NaN,NaN
3,94608,Grade 6 Science: Unit 2 Traits and Reproduction,186E87F2-A70F-11E2-85F4-D4629DFF4B22,MS-LS3-1,Develop and use a model to describe why struct...,601762,Marvin,Pineda,Moises,2023-11-13,...,6,67,2,Approaching Expectations,f,224,Standard,2024-12-16,NaN,NaN
4,94608,Grade 6 Science: Unit 2 Traits and Reproduction,186E87F2-A70F-11E2-85F4-D4629DFF4B22,MS-LS3-1,Develop and use a model to describe why struct...,44433,Nickolas,Perez,NaN,2023-11-13,...,6,33,1,Does Not Meet Expectations,f,224,Standard,2024-12-16,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141720,83107,Into Reading Grade 4 Module 3 Assessment,NaN,percent,NaN,602007,Cristopher,Oviedo Aguilar,Joel,2023-12-07,...,14,0,1,Does Not Meet Expectations,f,197,No_Standard,2024-12-16,1.0,Version 1
141721,83107,Into Reading Grade 4 Module 3 Assessment,NaN,percent,NaN,601837,Vanessa,Vargas-Paz,NaN,2023-12-07,...,14,0,1,Does Not Meet Expectations,f,197,No_Standard,2024-12-16,1.0,Version 1
141722,83107,Into Reading Grade 4 Module 3 Assessment,NaN,percent,NaN,601290,Dion,Tillman Jr,Lee,2024-01-08,...,14,29,1,Does Not Meet Expectations,f,197,No_Standard,2024-12-16,1.0,Version 1
141723,83107,Into Reading Grade 4 Module 3 Assessment,NaN,percent,NaN,601075,Isaiah,Baker Jr.,Kisean Mcelroy,2024-01-16,...,14,71,3,Meets Expectations,t,197,No_Standard,2024-12-16,1.0,Version 1


In [21]:
test_results_view['date_taken'].unique()

<DatetimeArray>
['2024-10-01 00:00:00', '2024-10-08 00:00:00', '2024-10-10 00:00:00',
 '2024-10-11 00:00:00', '2024-10-13 00:00:00', '2024-10-14 00:00:00',
 '2024-10-15 00:00:00', '2024-10-16 00:00:00']
Length: 8, dtype: datetime64[ns]

In [13]:
pd.read_csv('/home/g2015samtaylor/backups/illuminate/test_results_view.csv')

,assessment_id,year,date_taken,grade,local_student_id,test_type,curriculum,unit,unit_labels,title,standard_code,score,performance_band_level,performance_band_label,proficiency,mastered,__count,last_update
0,94608,23-24,2023-11-13,7.0,43939,unknown,Science,Unit 2,2.0,Grade 6 Science: Unit 2 Traits and Reproduction,MS-LS3-1,50,1,Does Not Meet Expectations,1 Does Not Meet Expectations,f,224,2024-12-16
1,94608,23-24,2023-11-13,7.0,43043,unknown,Science,Unit 2,2.0,Grade 6 Science: Unit 2 Traits and Reproduction,MS-LS3-1,67,2,Approaching Expectations,2 Approaching Expectations,f,224,2024-12-16
2,94608,23-24,2023-11-13,7.0,44433,unknown,Science,Unit 2,2.0,Grade 6 Science: Unit 2 Traits and Reproduction,MS-LS3-2,56,1,Does Not Meet Expectations,1 Does Not Meet Expectations,f,224,2024-12-16
3,94608,23-24,2023-11-13,7.0,601762,unknown,Science,Unit 2,2.0,Grade 6 Science: Unit 2 Traits and Reproduction,MS-LS3-1,67,2,Approaching Expectations,2 Approaching Expectations,f,224,2024-12-16
4,94608,23-24,2023-11-13,7.0,44433,unknown,Science,Unit 2,2.0,Grade 6 Science: Unit 2 Traits and Reproduction,MS-LS3-1,33,1,Does Not Meet Expectations,1 Does Not Meet Expectations,f,224,2024-12-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140132,83107,23-24,2023-12-07,5.0,602007,assessment,ELA,Module 3,3.0,Into Reading Grade 4 Module 3 Assessment,percent,0,1,Does Not Meet Expectations,1 Does Not Meet Expectations,f,197,2024-12-16
140133,83107,23-24,2023-12-07,5.0,601837,assessment,ELA,Module 3,3.0,Into Reading Grade 4 Module 3 Assessment,percent,0,1,Does Not Meet Expectations,1 Does Not Meet Expectations,f,197,2024-12-16
140134,83107,23-24,2024-01-08,5.0,601290,assessment,ELA,Module 3,3.0,Into Reading Grade 4 Module 3 Assessment,percent,29,1,Does Not Meet Expectations,1 Does Not Meet Expectations,f,197,2024-12-16
140135,83107,23-24,2024-01-16,5.0,601075,assessment,ELA,Module 3,3.0,Into Reading Grade 4 Module 3 Assessment,percent,71,3,Meets Expectations,3 Meets Expectations,t,197,2024-12-16


# Testing View Changes

In [ ]:
%load_ext autoreload
%autoreload 2

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
import logging
import os

test_results_combined = bring_together_test_results(test_results_no_standard, test_results_standard)
test_results_view = create_test_results_view(test_results_combined, '23-24') #add in grade level col, string matching

# Read in Raw Files

In [ ]:
import pandas as pd
c =pd.read_csv('/home/g2015samtaylor/illuminate/assessment_results_combined.csv')
v =pd.read_csv('/home/g2015samtaylor/illuminate/assessment_results_view.csv')
sr =pd.read_table('/home/icef/powerschool/Student_Rosters.txt')